In [1]:
import torch
from torch_geometric.data import Data
from neo4j import GraphDatabase
import numpy as np
import pandas as pd
import random, time, pickle, os

In [2]:
"""
Test script: Export one (apartment × POI class) subgraph from Neo4j
to a PyTorch Geometric Data object.

Steps:
1. Query apartment + POI nodes and edges from Neo4j.
2. Build node feature matrix (x), edge_index, edge_attr.
3. Package into torch_geometric.data.Data.
"""

# ---------------- Config ---------------- #
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "12345678"
DATABASE = "neo4jads"

# Radii per category (meters)
RADII = {"CERCA_DE_CAT1": 600.0, "CERCA_DE_CAT2": 1200.0, "CERCA_DE_CAT3": 2400.0}

# Example
TEST_APT_ID = 1548097259        # <-- replace with real id from your df
TEST_CLASS = "medical"    # one of the 11 classes

# ---------------- Cypher Queries ---------------- #
NODE_QUERY = """
MATCH (d:Departamento {id:$apt_id})
RETURN elementId(d) AS id, 'Departamento' AS label,
       d.id AS apt_id, d.latitude AS lat, d.longitude AS lon, null AS cat
UNION
MATCH (d:Departamento {id:$apt_id})-[r:CERCA_DE_CAT1|CERCA_DE_CAT2|CERCA_DE_CAT3]->(p:POI)
WHERE p.class = $class
RETURN elementId(p) AS id, 'POI' AS label,
       null AS apt_id, null AS lat, null AS lon, p.cat AS cat
"""

REL_QUERY = """
MATCH (d:Departamento {id:$apt_id})-[r:CERCA_DE_CAT1|CERCA_DE_CAT2|CERCA_DE_CAT3]->(p:POI)
WHERE p.class = $class
WITH elementId(d) AS source, elementId(p) AS target, type(r) AS t, r.distancia_metros AS dist
WITH source, target,
  CASE t
    WHEN 'CERCA_DE_CAT1' THEN CASE WHEN 1.0 - dist / $r_cat1 > 0.001 THEN 1.0 - dist / $r_cat1 ELSE 0.001 END
    WHEN 'CERCA_DE_CAT2' THEN CASE WHEN 1.0 - dist / $r_cat2 > 0.001 THEN 1.0 - dist / $r_cat2 ELSE 0.001 END
    ELSE CASE WHEN 1.0 - dist / $r_cat3 > 0.001 THEN 1.0 - dist / $r_cat3 ELSE 0.001 END
  END AS weight
RETURN source, target, weight
"""

# ---------------- Export Function ---------------- #
def export_apartment_class_graph(apt_id: int, class_name: str) -> Data:
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    with driver.session(database=DATABASE) as session:
        
        # --- Nodes
        nodes = session.run(
            NODE_QUERY,
            {"apt_id": apt_id, "class": class_name}
        ).data()
        if not nodes:
            print(f"No nodes found for apartment {apt_id}, class {class_name}")
            return None
        node_df = pd.DataFrame(nodes)
        
        # Map neo4j ids -> local indices
        id_map = {nid: i for i, nid in enumerate(node_df["id"].tolist())}
        
        # --- Edges
        edges = session.run(
            REL_QUERY,
            {
                "apt_id": apt_id, "class": class_name,
                "r_cat1": RADII["CERCA_DE_CAT1"],
                "r_cat2": RADII["CERCA_DE_CAT2"],
                "r_cat3": RADII["CERCA_DE_CAT3"],
            }
        ).data()
        if not edges:
            print(f"No edges found for apartment {apt_id}, class {class_name}")
            return None
        edge_df = pd.DataFrame(edges)
        
        # Remap to local indices
        edge_index = torch.tensor([
            [id_map[s] for s in edge_df["source"]],
            [id_map[t] for t in edge_df["target"]],
        ], dtype=torch.long)
        edge_attr = torch.tensor(edge_df["weight"].values, dtype=torch.float).unsqueeze(1)
        
        # --- Node features
        # Apartment node = mark explicitly
        feats = []
        for _, row in node_df.iterrows():
            if row["label"] == "Departamento":
                feats.append([1.0, 0.0, 0.0, 0.0])  # [is_apartment, cat1, cat2, cat3]
            else:
                onehot = [0.0, 0.0, 0.0, 0.0]
                if row["cat"] is not None:
                    try:
                        idx = int(row["cat"])
                        if 0 <= idx < len(onehot):
                            onehot[idx] = 1.0
                    except (ValueError, TypeError):
                        pass  # skip if cat is invalid
                feats.append(onehot)
        x = torch.tensor(np.array(feats), dtype=torch.float)
        
        # Build Data object
        data = Data(
            x=x,
            edge_index=edge_index,
            edge_attr=edge_attr,
            apt_id=apt_id,
            poi_class=class_name
        )
        return data



In [3]:
df_deptos = pd.read_csv('Datasets/dataset_final.csv')

In [4]:
# ---------------- Test ---------------- #
CLASSES = [
    'sport_and_leisure',
    'medical',
    'education_prim',
    'veterinary',
    'food_and_drink_stores',
    'arts_and_entertainment',
    'food_and_drink',
    'park_like',
    'security',
    'religion',
    'education_sup'
]


In [7]:

#
OUTPUT_FILE = "apartment_graphs.pkl"
BATCH_SIZE = 100   # change this number freely

# Load progress if exists
if os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, "rb") as f:
        all_graphs = pickle.load(f)
    done_ids = set(all_graphs.keys())
    print(f"Resuming: {len(done_ids)} apartments already done.")
else:
    all_graphs = {}
    done_ids = set()

# All apartments
apt_ids = df_deptos["id"].tolist()
pending_ids = [i for i in apt_ids if i not in done_ids]

# Limit to one batch
batch_ids = pending_ids[:BATCH_SIZE]
print(f"Processing {len(batch_ids)} apartments in this batch "
      f"(remaining after this: {len(pending_ids)-len(batch_ids)})")

for idx, apt_id in enumerate(batch_ids, 1):
    print(f"\n[{idx}/{len(batch_ids)}] Apartment ID={apt_id}")
    start = time.time()

    graphs = {}
    for cls in CLASSES:
        g = export_apartment_class_graph(apt_id, cls)
        graphs[cls] = g  # can be None for now

    all_graphs[apt_id] = graphs

    # Save progress after each apartment
    with open(OUTPUT_FILE, "wb") as f:
        pickle.dump(all_graphs, f)

    print(f"Done apt {apt_id} in {time.time()-start:.2f} sec "
          f"(progress: {len(all_graphs)}/{len(apt_ids)})")

print("✅ Batch completed and saved.")


Resuming: 4810 apartments already done.
Processing 100 apartments in this batch (remaining after this: 20305)

[1/100] Apartment ID=2807806402
Done apt 2807806402 in 39.79 sec (progress: 4811/25215)

[2/100] Apartment ID=1571492613
Done apt 1571492613 in 39.57 sec (progress: 4812/25215)

[3/100] Apartment ID=2803722742
Done apt 2803722742 in 39.87 sec (progress: 4813/25215)

[4/100] Apartment ID=2863686322
Done apt 2863686322 in 39.84 sec (progress: 4814/25215)

[5/100] Apartment ID=2857003382
Done apt 2857003382 in 40.18 sec (progress: 4815/25215)

[6/100] Apartment ID=2856965500
Done apt 2856965500 in 39.56 sec (progress: 4816/25215)

[7/100] Apartment ID=2834769252
Done apt 2834769252 in 39.78 sec (progress: 4817/25215)

[8/100] Apartment ID=2834717554
Done apt 2834717554 in 39.16 sec (progress: 4818/25215)

[9/100] Apartment ID=1590264475
Done apt 1590264475 in 39.04 sec (progress: 4819/25215)

[10/100] Apartment ID=1571546745
Done apt 1571546745 in 39.74 sec (progress: 4820/25215)

In [16]:

with open("apartment_graphs.pkl", "rb") as f:
    graphs = pickle.load(f)

print(f"Loaded {len(graphs)} apartments")

# Print first 3 keys (apartment IDs)
print("Some apartment IDs:", list(graphs.keys())[:3])

# Inspect one apartment
apt_id = list(graphs.keys())[0]
apt_data = graphs[apt_id]

print(f"\nApartment {apt_id}:")
for cls, data in apt_data.items():
    if data is None:
        print(f"  {cls}: None (no POIs)")
    else:
        print(f"  {cls}: Data(x={data.x.shape}, edge_index={data.edge_index.shape}, edge_attr={data.edge_attr.shape})")


Loaded 10 apartments
Some apartment IDs: [1548097259, 2732119230, 1592811585]

Apartment 1548097259:
  sport_and_leisure: Data(x=torch.Size([90, 4]), edge_index=torch.Size([2, 89]), edge_attr=torch.Size([89, 1]))
  medical: Data(x=torch.Size([32, 4]), edge_index=torch.Size([2, 31]), edge_attr=torch.Size([31, 1]))
  education_prim: Data(x=torch.Size([9, 4]), edge_index=torch.Size([2, 8]), edge_attr=torch.Size([8, 1]))
  veterinary: Data(x=torch.Size([2, 4]), edge_index=torch.Size([2, 1]), edge_attr=torch.Size([1, 1]))
  food_and_drink_stores: Data(x=torch.Size([13, 4]), edge_index=torch.Size([2, 12]), edge_attr=torch.Size([12, 1]))
  arts_and_entertainment: Data(x=torch.Size([26, 4]), edge_index=torch.Size([2, 25]), edge_attr=torch.Size([25, 1]))
  food_and_drink: Data(x=torch.Size([26, 4]), edge_index=torch.Size([2, 25]), edge_attr=torch.Size([25, 1]))
  park_like: Data(x=torch.Size([7, 4]), edge_index=torch.Size([2, 6]), edge_attr=torch.Size([6, 1]))
  security: Data(x=torch.Size([8, 

In [18]:
import pickle
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, global_mean_pool

# ---------------- Load the pickle file ---------------- #
with open("apartment_graphs.pkl", "rb") as f:
    apartment_graphs = pickle.load(f)

# ---------------- Small GNN model ---------------- #
class SimpleGraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index, edge_attr=None, batch=None):
        # Two-layer GraphSAGE
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        if batch is None:
            # Single graph → make batch of zeros
            batch = torch.zeros(x.size(0), dtype=torch.long)
        return global_mean_pool(x, batch)  # → one vector per graph


# ---------------- Pick one apartment ---------------- #
apt_id = next(iter(apartment_graphs.keys()))  # just the first apartment
graphs = apartment_graphs[apt_id]

# Feature dimension (4 in your case: is_apartment, cat1, cat2, cat3)
in_channels = graphs[next(iter(graphs))].x.shape[1]
model = SimpleGraphSAGE(in_channels, hidden_channels=16, out_channels=8)

# ---------------- Compute embeddings ---------------- #
embeddings = {}
for cls, g in graphs.items():
    if g is None:
        embeddings[cls] = None
    else:
        with torch.no_grad():
            emb = model(g.x, g.edge_index)
            embeddings[cls] = emb.squeeze().numpy()  # shape (out_channels,)

# ---------------- Inspect ---------------- #
print(f"Apartment {apt_id} embeddings:")
for cls, emb in embeddings.items():
    print(cls, None if emb is None else emb)


Apartment 1548097259 embeddings:
sport_and_leisure [-0.18648088  0.5579107  -0.4131222   0.27406204 -0.35570735  0.40606898
  0.14326589 -0.5241446 ]
medical [-0.20881331  0.31410986 -0.40781352  0.36022988 -0.28139758  0.50777537
  0.04728226 -0.36782062]
education_prim [-0.34600687  0.42414403 -0.4898538   0.15145272 -0.4337186   0.6039778
  0.08607332 -0.5473764 ]
veterinary [-0.27915993  0.22826114 -0.191151    0.27866125 -0.18302271  0.27510607
  0.21248034 -0.57624197]
food_and_drink_stores [-0.27390623  0.4828838  -0.44862312  0.20662722 -0.39463884  0.5065929
  0.11644609 -0.5412748 ]
arts_and_entertainment [-0.17523065  0.2991605  -0.37270257  0.40328738 -0.24081656  0.4592786
  0.05530962 -0.3462455 ]
food_and_drink [-0.32072455  0.48084846 -0.51296467  0.15983051 -0.4509474   0.6042404
  0.08255335 -0.5402578 ]
park_like [-0.34960094  0.40333438 -0.4733036   0.15413487 -0.42034778  0.59179056
  0.09157699 -0.5501465 ]
security [-0.20843989  0.20837197 -0.3466561   0.4006459 